# **Generador de clasificador de texto en categorías binarias no excluyentes a partir de un dataset**:

Esta consola tiene como objetivo generar un multiclasificador de texto de categorías binarias. Por ejemplo, recibir un texto plano y clasificar entre True o False en distintas categorías, no excluyentes entre sí. Por ejemplo: True en dos o tres, y False en el resto.

La función principal que se debe modificar es la del algoritmo.

**Nota**: Es importante en la descripción dar información respecto del algoritmo utilizado, ya que una vez entrenado el mismo, es imposible obtener para atrás su información.

El objetivo final es generar un archivo exportado de pickle (llamado, por ejemplo, {nombre}_{version}.algo) que contendrá la siguiente información:
1. name: Contiene el nombre del algoritmo.
1. description: Contiene una descripción de este algoritmo.
1. observations: Cualquier otra información que se quiera colocar sobre el algoritm.
1. date: Fecha en la cual se guardó el algoritmo.
1. type: Tipo indicador del algoritmo (que haga referencia a que se hizo con este método).
1. classifiers: Un diccionario que contiene el predictor para cada label. Por ejemplo, classifiers\["notes"\].
1. classification_stats: Un diccionario como el de classifiers, pero que contiene para cada label un diccionario con: count, positive_count, negative_count, accuracy, true_positives, true_negatives, false_positives y false_negatives.
1. sample_training_stats: Datos con como fue utilizado el dataframe: test_size, random_state y fecha (o sea, cómo se obtuvo el sample de entrenamiento).
1. dataset_information: Contiene toda la información necesaria del dataset.


Por su parte, la información del dataset contendrá lo siguiente (es información del dataset, sin incluir el dataframe, por cuestiones de tamaño):
1. name: Contiene el nombre del dataset utilizado.
1. file_path: Contiene la ubicación (path) que tenía el dataset utilizado.
1. descripcion: Contiene la descripción del dataset.
1. date: Contiene la fecha del dataset.
1. shape: Contiene el dataframe.shape
1. original_labels: Contiene el dataframe.keys()
1. text_label: Contiene cuál es la etiqueta que contiene el texto.
1. observations: Contiene las observaciones del dataset.

Todo este contenido será exportado con pickle, de forma que podrá ser importado por pickle del mismo modo.

## Código
### Importo librerías necesarias

In [17]:
import datetime
import nltk
import numpy
import os
import pandas
import pickle
import re
import string
import unicodedata

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn import metrics

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\licma\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

### Declarar constantes importantes
Es importante declarar todas, leer la definición arriba.

In [18]:
# Ubicación del archivo dataset que se utilizará para hacer el entrenamiento.
DATASET_FILE_PATH = os.path.abspath('dataset.data')
# Ubicación de la carpeta en la cual se guardará el algoritmo. 
ALGORITHM_TARGET_DIR_PATH = os.path.dirname(os.path.abspath('__file__'))

# Datos propios del algoritmo (coinciden con la explicación de arriba): ver nombre, versión, descripción y observaciones.
ALGORITHM_NAME=r'Tesis'
ALGORITHM_VERSION=r'1'
ALGORITHM_DESCRIPTION=r'Prueba del A20NewsGroups'
ALGORITHM_OBSERVATIONS=r'Prueba ejecutadda con el A20NewsGroups'

# NO MODIFICAR ESTE.
ALGORITHM_TYPE="text_binary_classification"


### Obtener el contenido de un archivo con pickle.

In [19]:
def load_with_pickle(file_path):
    file = open(file_path, 'rb')
    file_content = file.read()
    file.close()
    return pickle.loads(file_content)

### Guardar el contenido de un archivo con pickle.

In [20]:
def export_with_pickle(file_path, object_to_save):
    file = open(file_path, 'wb')
    picklestring = pickle.dumps(object_to_save, protocol=pickle.HIGHEST_PROTOCOL)
    file.write(picklestring)
    file.close()

### Limpiador de texto
Limpia los textos de, por ejemplo, el nombre del algoritmo y de la versión.

In [21]:
def clean_text(text):
    # Remove special chars.
    text_nfkd_form = unicodedata.normalize('NFKD', text)
    text = u"".join([c for c in text_nfkd_form if not unicodedata.combining(c)])
    
    # Only keep this chars.
    PERMITTED_CHARS = "0123456789abcdefghijklmnopqrstuvwxyz_-. " 
    text = text.lower()
    text = "".join(c for c in text if c in PERMITTED_CHARS)

    # Replace spaces with underscore.
    text = text.replace(" ", "_")
    return text

### Preprocesar texto
Es importante notar que esta función prepara el texto plano para ser metido dentro del algoritmo. Es importante que el que utilice alguno de los clasificadores use esta misma función antes de enviar el texto al algoritmo.

In [22]:
def preprocess_text(text):
    text = text.lower()
    # Remove punctuation
    text = "".join([i for i in text if i not in string.punctuation])

    # Remove anything but alphanumeric words.
    text = text.replace('ñ', 'ni')
    text = re.sub(r'\W', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    text = text.strip()

    # Remove accents:
    text_nfkd_form = unicodedata.normalize('NFKD', text)
    text = u"".join([c for c in text_nfkd_form if not unicodedata.combining(c)])

    # Remove stopwords
    text_words = text.split(' ')
    stop_words = nltk.corpus.stopwords.words('english')
    text_words = [i for i in text_words if i not in stop_words]
 
    # Remove very long words, they must be OCR errors.
    text_words = [i for i in text_words if len(i) < 15]
    
    # Remove single chars text words.
    text_words = [i for i in text_words if len(i) > 1]
    return " ".join(text_words)

### Cargar dataset
Obtiene el dataset del archivo de dataset especificado y preprocesa sus textos.

In [23]:
def load_dataset(dataset_file_path):
    dataset = load_with_pickle(dataset_file_path)
    required_properties = ["name","description","dataframe","date","observations","text_label"]
    dataset_properties = dataset.keys()
    for required_property in required_properties:
        if required_property not in dataset_properties:
            raise Exception(f'El dataset no contiene la propiedad requerida "{required_property}"."')
    text_label = dataset["text_label"]
    dataset["dataframe"][text_label] = dataset["dataframe"][text_label].apply( lambda data: preprocess_text(data))
    return dataset 

### Obtener información del dataset.
Recupera la información del dataset y devuelve un diccionario para colocar como información del algoritmo.

In [24]:
def get_dataset_information(dataset, dataset_file_path):
    dataframe = dataset["dataframe"]
    return {
        "name": dataset["name"],
        "file_path": dataset_file_path,
        "description": dataset["description"],
        "date": dataset["date"],
        "shape": dataframe.shape,
        "original_labels": dataframe.keys(),
        "text_label": dataset["text_label"],
        "observations": dataset["observations"]        
    }

### Obtener las estadísticas de un clasificador binario.
Recibe un clasificador entrenado, y el conjunto de entrenaimento y de test, para obtener sus estadísticas (cantidad, efectividad, falsos negativos, positivos, etc.)

In [25]:
def get_binary_classifier_stats(classifier, x_train, y_train, x_test, y_test):
    predicted = classifier.predict(x_test)
    accuracy = numpy.mean(predicted == y_test)
    results = {}
    results["count"] = len(y_test)
    results["test_sample_positive_count"] = numpy.sum(y_test == 1)
    results["test_sample_negative_count"] = numpy.sum(y_test == 0)
    results["predicted_positive_count"] = numpy.sum(predicted == 1)
    results["predicted_negative_count"] = numpy.sum(predicted == 0)
    results["accuracy"] = accuracy
    if accuracy == 1:
        results["true_positives"] = results['test_sample_positive_count']
        results['true_negatives'] = results['test_sample_negative_count']
        results['false_positives'] = 0
        results['false_negatives'] = 0
        return results
    confusion_matrix_results = confusion_matrix(predicted, y_test)
    results["true_positives"] = confusion_matrix_results[1][1]
    results['true_negatives'] = confusion_matrix_results[0][0]
    results['false_positives'] = confusion_matrix_results[1][0]
    results['false_negatives'] = confusion_matrix_results[0][1]
    results['predicted_positive_accuracy'] = results['true_positives'] / results['test_sample_positive_count']
    results['predicted_negative_accuracy'] = results['true_negatives'] / results['test_sample_negative_count']
    return results

### Imprimir en pantalla las estadísticas de un clasificador binario.
Recibe un clasificador entrenado, y el conjunto de entrenaimento y de test, para obtener sus estadísticas (cantidad, efectividad, falsos negativos, positivos, etc.) y lo imprime en pantalla

In [26]:
def print_binary_classifier_stats(classifier, x_train, y_train, x_test, y_test):
    stats = get_binary_classifier_stats(classifier, x_train, y_train, x_test, y_test)
    print(f'- Count: {stats["count"]}')
    print(f'- Test sample positive count: {stats["test_sample_positive_count"]}')
    print(f'- Test sample negative count: {stats["test_sample_negative_count"]}')
    print(f'- Predicted positive count: {stats["predicted_positive_count"]}')
    print(f'- Predicted segative count: {stats["predicted_negative_count"]}')
    print(f'- True positives: {stats["true_positives"]}')
    print(f'- True negatives: {stats["true_negatives"]}')
    print(f'- False positives: {stats["false_positives"]}')
    print(f'- False negatives: {stats["false_negatives"]}')
    print(f'- Accuracy: {stats["accuracy"]}')
    print(f'- Positives accuracy: {stats["predicted_positive_accuracy"]}')
    print(f'- Negatives accuracy: {stats["predicted_negative_accuracy"]}')

### Importante:
### *Generar el clasificador binario*
Aquí se generan los distintos clasificadores binarios. Recibe un conjunto de entrenamiento y sus correspondientes etiquetas. Esta función será utilizada para generar los distintos clasificadores binarios. Esta función será modificada para poder mejorar un algoritmo. Es importante documentar bien lo realizado en la descripción, ya que no es posible obtener información del algoritmo al final para poner en el diccionario de información.

In [52]:
# CLASSIFIER
def generate_binary_classifier(train_x, train_y, random_state = 42, max_iter = 100):
#    naive_bayes = MultinomialNB()
    sgd_classifier = SGDClassifier(loss='hinge', penalty='l2',
                          alpha=1e-3, random_state=random_state,
                          max_iter=max_iter, tol=None)
    classifier = Pipeline([
        ('vect', CountVectorizer()),
        ('tfidf', TfidfTransformer()),
        ('clf', sgd_classifier), # naive_bayes),
    ])
    classifier.fit(train_x, train_y)
    return classifier

### Generar los múltiples clasificadores a partir de un dataset
Recibe un dataset e información de cómo obtener sus muestras (tamaño de la muestra, semilla de aleatoriedad, que etiquetas entrenar, etc.) y devuelve los distintos clasificadores.

Si no recibe qué etiquetas entrenar, entrena todas.

Si el valor "verbose" vale True, imprime en pantalla las estadísticas de los diferentes clasificadores.

In [55]:
def generate_binary_classifiers_from_dataset(dataset, test_size, random_state, labels_to_train = [], verbose = True, iterations=30):
    dataframe = dataset["dataframe"]
    text_label = dataset["text_label"]
    train, test = train_test_split(dataframe, random_state=random_state, test_size=test_size, shuffle=True)
    stats = []
    if len(labels_to_train) == 0:
        labels_to_train = dataframe.keys()
    classifiers = {} 
    classifiers_stats = {} 
    for label_to_train in labels_to_train:
        if label_to_train == text_label:
            continue
        new_classifier = generate_binary_classifier(train[text_label], train[label_to_train], random_state, iterations)
        classifiers[label_to_train] = new_classifier
        classifiers_stats[label_to_train] = get_binary_classifier_stats(new_classifier, train[text_label], train[label_to_train], test[text_label], test[label_to_train])
        if verbose == True:
            print(f'\nStats de atributo "{label_to_train}":')
            print_binary_classifier_stats(new_classifier, train[text_label], train[label_to_train], test[text_label], test[label_to_train])
    return classifiers, classifiers_stats

### Clasificar con los clasificadores
Permite obtener qué clasificadores dan positivo en el dataset seleccionado. Sirve para testear.

In [56]:
def classify_with_binary_classifiers(classifiers, text, labels_to_get = []):
    if len(labels_to_get) == 0:
        labels_to_get = classifiers.keys()
    results = []
    text = preprocess_text(text)
    for label_to_get in labels_to_get:
        label_result = classifiers[label_to_get].predict([text])
        if label_result[0] == 1:
            results.append(label_to_get)
    return results

### Generar el archivo con los algoritmos.
Genera el archivo {nombre}\_{version}.algo con la información especificada más arriba. Devuelve el nombre del archivo creado.

In [69]:
def generate_binary_classifiers_file(target_dir_path, dataset_file_path, test_size, random_state, labels_to_train = []):
    dataset = load_dataset(dataset_file_path)
    dataframe = dataset["dataframe"]
    text_label = dataset["text_label"]
    train, test = train_test_split(dataframe, random_state=random_state, test_size=test_size, shuffle=True)
    classifiers, _stats =  generate_binary_classifiers_from_dataset(dataset, test_size, random_state, labels_to_train, False)
    labels = classifiers.keys()
    classification_stats = {}
    for label in labels:
        label_classifier = classifiers[label]
        label_stats = get_binary_classifier_stats(label_classifier, train[text_label], train[label], test[text_label], test[label])
        classification_stats[label] = label_stats
    dataset_information = get_dataset_information(dataset, dataset_file_path)
    date = datetime.datetime.now()
    sample_training_stats = { "random_state": random_state, "test_size": test_size, "date": date }    
    file_content = {
        "name": ALGORITHM_NAME,
        "description": ALGORITHM_DESCRIPTION,
        "observations": ALGORITHM_OBSERVATIONS,
        "date": date,
        "type": ALGORITHM_TYPE,
        "classifiers": classifiers, 
        "classification_stats": classification_stats,
        "sample_training_stats": sample_training_stats,
        "dataset_information": dataset_information,
    }
    final_file_name = f'{clean_text(ALGORITHM_NAME)}_v{clean_text(ALGORITHM_VERSION)}.algo'
    final_file_path = os.path.join(target_dir_path, final_file_name)
    export_with_pickle(final_file_path, file_content)
    return final_file_path

### Cargar el dataset.
Carga el dataset y lo muestra en pantalla para probar que funciona correctamente.

In [58]:
dataset = load_dataset(DATASET_FILE_PATH)
dataframe = dataset["dataframe"]
display(dataframe)

,data,deportes,electronica,espacio,medicina,politica,religion,tecnologia,vehiculos
0,logistician subject 77s organization worcester...,1,0,0,0,0,0,0,0
1,dxf12pocwruedu douglas fowler subject atas nl ...,1,0,0,0,0,0,0,0
2,dxf12pocwruedu douglas fowler subject 1dimensi...,1,0,0,0,0,0,0,0
3,scottytissue subject 15day 30day 60day disable...,1,0,0,0,0,0,0,0
4,matthew militzok subject 1992 1993 final nhl p...,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...
8350,tom neumann subject vw passat replyto tom neum...,0,0,0,0,0,0,0,1
8351,dave chu subject wanted opinions 75 mg organiz...,0,0,0,0,0,0,0,1
8352,tommy szeto subject water trunk 89 probe organ...,0,0,0,0,0,0,0,1
8353,terry quinn subject waxing new car organizatio...,0,0,0,0,0,0,0,1


### Establecer variables para el dataset de entrenamiento
Las mismas serán las variables correspondientes a: tamaño de la muestra de testeo( por ejemplo, 0.25 será un 25% de muestra para test y 75% para entrenamiento), semilla de aleatoriedad, etiquetas para entrenar (cuáles del dataset serán entrenadas, si se deja vacío, con todas) y si se mostrará en pantalla la distinta información.

In [65]:
SAMPLE_TEST_SIZE = 0.2
RANDOM_STATE = 42
LABELS_TO_TRAIN = [] # if empty, all will be trained.
VERBOSE = True
ITERATIONS=50

### Generar los distintos clasificadores.
Genera los distintos clasificadores con los parámetros establecidos. Conviene poner "True" en Verbose para poder ver los distintos stats que obtiene cada clasificador.

In [66]:
classifiers, classifiers_stats = generate_binary_classifiers_from_dataset(dataset, SAMPLE_TEST_SIZE, RANDOM_STATE, LABELS_TO_TRAIN, VERBOSE, ITERATIONS)


Stats de atributo "deportes":
- Count: 1671
- Test sample positive count: 228
- Test sample negative count: 1443
- Predicted positive count: 149
- Predicted segative count: 1522
- True positives: 149
- True negatives: 1443
- False positives: 0
- False negatives: 79
- Accuracy: 0.952722920406942
- Positives accuracy: 0.6535087719298246
- Negatives accuracy: 1.0

Stats de atributo "electronica":
- Count: 1671
- Test sample positive count: 115
- Test sample negative count: 1556
- Predicted positive count: 1
- Predicted segative count: 1670
- True positives: 1
- True negatives: 1556
- False positives: 0
- False negatives: 114
- Accuracy: 0.9317773788150808
- Positives accuracy: 0.008695652173913044
- Negatives accuracy: 1.0

Stats de atributo "espacio":
- Count: 1671
- Test sample positive count: 83
- Test sample negative count: 1588
- Predicted positive count: 20
- Predicted segative count: 1651
- True positives: 20
- True negatives: 1588
- False positives: 0
- False negatives: 63
- Accu

# Obtener los mejores stats:

In [63]:
def get_best_stats(dataset):
    min_random_state = 1
    max_random_state = 100
    min_sample_test_size = 20
    max_sample_test_size = 30
    best_accuracy = 0
    best_random_state = min_random_state
    best_sample_test_size = min_sample_test_size
    count = 1
    for temp_random_state in range(min_random_state, max_random_state):
        for temp_sample_test_size in range(min_sample_test_size, max_sample_test_size):
            temp_sample_test_size = temp_sample_test_size / 100
            classifiers, classifiers_stats = generate_binary_classifiers_from_dataset(dataset, temp_sample_test_size, temp_random_state, LABELS_TO_TRAIN, False)
            positive_accuracies = [value['predicted_positive_accuracy'] for value in classifiers_stats.values()]
            average_positive_accuracy = sum(positive_accuracies) / len(positive_accuracies) if positive_accuracies else 0
            if average_positive_accuracy > best_accuracy:
                best_accuracy = average_positive_accuracy
                best_random_state = temp_random_state
                best_sample_test_size = temp_sample_test_size
            print(f'Test {count}: rnd state: {temp_random_state}, test size: {temp_sample_test_size}, acc: {average_positive_accuracy}. Mejor: rnd state {best_random_state}, test size: {best_sample_test_size}, acc: {best_accuracy}')
            count = count + 1
    print(f'Mejores stats: rnd state: rnd state {best_random_state}, test size: {best_sample_test_size}, acc: {best_accuracy}')


In [64]:
get_best_stats(dataset)

Test 1: rnd state: 1, test size: 0.2, acc: 0.32187266996307784. Mejor: rnd state 1, test size: 0.2, acc: 0.32187266996307784
Test 2: rnd state: 1, test size: 0.21, acc: 0.3202100375391053. Mejor: rnd state 1, test size: 0.2, acc: 0.32187266996307784
Test 3: rnd state: 1, test size: 0.22, acc: 0.3222235837751096. Mejor: rnd state 1, test size: 0.22, acc: 0.3222235837751096
Test 4: rnd state: 1, test size: 0.23, acc: 0.3256325761280356. Mejor: rnd state 1, test size: 0.23, acc: 0.3256325761280356
Test 5: rnd state: 1, test size: 0.24, acc: 0.3277461027490905. Mejor: rnd state 1, test size: 0.24, acc: 0.3277461027490905
Test 6: rnd state: 1, test size: 0.25, acc: 0.3262784868502862. Mejor: rnd state 1, test size: 0.24, acc: 0.3277461027490905
Test 7: rnd state: 1, test size: 0.26, acc: 0.3239901846406538. Mejor: rnd state 1, test size: 0.24, acc: 0.3277461027490905
Test 8: rnd state: 1, test size: 0.27, acc: 0.3219580145011355. Mejor: rnd state 1, test size: 0.24, acc: 0.3277461027490905


### Testear clasificaciones
Este bloque de código permite poner un texto libre para ver qué devuelve como output el algoritmo. Es una forma de testear

In [67]:
text_categories = classify_with_binary_classifiers(classifiers, "vehicle")
print(text_categories)

[]


### Genera el archivo final con el algoritmo
Exporta con pickle el algoritmo y devuelve dónde fue guardado.

In [70]:
algorithm_saved_path = generate_binary_classifiers_file(ALGORITHM_TARGET_DIR_PATH, DATASET_FILE_PATH, SAMPLE_TEST_SIZE, RANDOM_STATE, LABELS_TO_TRAIN ) 
algorithm_saved_path

'C:\\Users\\licma\\Desktop\\repositorios\\siglo21-tesis-lic-en-informatica\\algorithms crafting\\tesis_v1.algo'

### Muestra el contenido del último archivo guardado

In [71]:
load_with_pickle(algorithm_saved_path)

{'name': 'Tesis',
 'description': 'Prueba del A20NewsGroups',
 'observations': 'Prueba ejecutadda con el A20NewsGroups',
 'date': datetime.datetime(2023, 12, 19, 16, 32, 48, 104748),
 'type': 'text_binary_classification',
 'classifiers': {'deportes': Pipeline(steps=[('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                  ('clf',
                   SGDClassifier(alpha=0.001, max_iter=30, random_state=42,
                                 tol=None))]),
  'electronica': Pipeline(steps=[('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                  ('clf',
                   SGDClassifier(alpha=0.001, max_iter=30, random_state=42,
                                 tol=None))]),
  'espacio': Pipeline(steps=[('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                  ('clf',
                   SGDClassifier(alpha=0.001, max_iter=30, random_state=42,
                                 tol=None))]),
  'medicina': Pipeline(steps=[('vect', Count